## MySQL

In [ ]:
!pip install install mysql-connector-python
!pip install mysql

In [ ]:
host=DB_HOST
port=DB_PORT
username= DB_USESRNAME
password= DB_PASSWORD
database=DB_database

print(f"mysql://{username}:{password}@{host}:{port}/{database}")

In [ ]:
import mysql.connector

# MySQL database credentials
host='dockerlab.westeurope.cloudapp.azure.com'
port=3306
username='SP_2'
password='WEUTm9xuS12pH3-gG8obLIozeZi2A8gphHzdAH'
database='SP_2'


# Establish a connection to the MySQL database
cnx = mysql.connector.connect(user=username, password=password,
                              host=host, database=database)
cursor = cnx.cursor()

# SQL statements for creating tables in MySQL
create_table_statements = [
    """CREATE TABLE IF NOT EXISTS Patient (
        patient_id INT AUTO_INCREMENT PRIMARY KEY,
        full_name VARCHAR(255),
        date_of_birth DATE,
        address TEXT,
        gender VARCHAR(255),
        contact_number VARCHAR(255)
    )""",
    """CREATE TABLE IF NOT EXISTS Subscriber (
        subscriber_id INT AUTO_INCREMENT PRIMARY KEY,
        name VARCHAR(255),
        type VARCHAR(255),
        contact_details TEXT
    )""",
    """CREATE TABLE IF NOT EXISTS Manufacturer (
        manufacturer_id INT AUTO_INCREMENT PRIMARY KEY,
        manufacturer_name VARCHAR(255),
        manufacturer_address TEXT,
        manufacturer_contact VARCHAR(255)
    )""",
    """CREATE TABLE IF NOT EXISTS DeviceStaticInfo (
        device_id INT AUTO_INCREMENT PRIMARY KEY,
        manufacturer_id INT,
        model VARCHAR(255),
        purchase_date DATE,
        warranty_expiry DATE,
        FOREIGN KEY (manufacturer_id) REFERENCES Manufacturer(manufacturer_id)
    )""",
    """CREATE TABLE IF NOT EXISTS DeviceOperationalData (
        data_id INT AUTO_INCREMENT PRIMARY KEY,
        device_id INT,
        timestamp TIMESTAMP,
        battery_level INT,
        firmware_version VARCHAR(255),
        connectivity_status VARCHAR(255),
        error_codes VARCHAR(255),
        FOREIGN KEY (device_id) REFERENCES DeviceStaticInfo(device_id)
    )""",
    """CREATE TABLE IF NOT EXISTS GlucoseReadings (
        reading_id INT AUTO_INCREMENT PRIMARY KEY,
        patient_id INT,
        device_id INT,
        glucose_level FLOAT,
        timestamp TIMESTAMP,
        location TEXT,
        FOREIGN KEY (patient_id) REFERENCES Patient(patient_id),
        FOREIGN KEY (device_id) REFERENCES DeviceStaticInfo(device_id)
    )""",
    """CREATE TABLE IF NOT EXISTS Alerts (
        alert_id INT AUTO_INCREMENT PRIMARY KEY,
        patient_id INT,
        reading_id INT,
        alert_type VARCHAR(255),
        message TEXT,
        timestamp TIMESTAMP,
        FOREIGN KEY (patient_id) REFERENCES Patient(patient_id),
        FOREIGN KEY (reading_id) REFERENCES GlucoseReadings(reading_id)
    )""",
    """CREATE TABLE IF NOT EXISTS AlertLifecycle (
        lifecycle_id INT AUTO_INCREMENT PRIMARY KEY,
        alert_id INT,
        action_by_user_id INT,
        action_type VARCHAR(255),
        action_note TEXT,
        action_timestamp TIMESTAMP,
        FOREIGN KEY (alert_id) REFERENCES Alerts(alert_id),
        FOREIGN KEY (action_by_user_id) REFERENCES Subscriber(subscriber_id)
    )""",
    """CREATE TABLE IF NOT EXISTS SubscribedPatients (
        subscriber_id INT,
        patient_id INT,
        PRIMARY KEY (subscriber_id, patient_id),
        FOREIGN KEY (subscriber_id) REFERENCES Subscriber(subscriber_id),
        FOREIGN KEY (patient_id) REFERENCES Patient(patient_id)
    )"""
]

# Execute each SQL statement to create the tables
for statement in create_table_statements:
    cursor.execute(statement)

# Commit the changes to the database
cnx.commit()

# Close the cursor and the connection
cursor.close()
cnx.close()


## Popualate Database with synthetic data

In [ ]:
from faker import Faker
import sqlite3

fake = Faker()


# Connect to MySQL database
conn = mysql.connector.connect(
    host=db_host,
    user=db_user,
    password=db_password,
    database=db_name
)
cursor = conn.cursor()


# Generate data for Patient table
for _ in range(10):  # Assuming you want to generate data for 10 patients
    full_name = fake.name()
    gender = fake.random_element(elements=['Male', 'Female', 'Other'])
    date_of_birth = fake.date_of_birth().strftime('%Y-%m-%d')
    address = fake.address()
    contact_number = fake.phone_number()
    cursor.execute("INSERT INTO Patient (full_name, date_of_birth, address, gender, contact_number) VALUES (?, ?, ?, ?, ?)",
                   (full_name, date_of_birth, address, gender, contact_number))

# Generate data for Subscriber table
for _ in range(5):  # Assuming you want to generate data for 5 subscribers
    name = fake.name()
    subscriber_type = fake.random_element(elements=['patient', 'doctor', 'caregiver', 'app'])
    contact_details = fake.email()
    cursor.execute("INSERT INTO Subscriber (name, type, contact_details) VALUES (?, ?, ?)",
                   (name, subscriber_type, contact_details))

# Generate data for Manufacturer table
for _ in range(3):  # Assuming you want to generate data for 3 manufacturers
    manufacturer_name = fake.company()
    manufacturer_address = fake.address()
    manufacturer_contact = fake.phone_number()
    cursor.execute("INSERT INTO Manufacturer (manufacturer_name, manufacturer_address, manufacturer_contact) VALUES (?, ?, ?)",
                   (manufacturer_name, manufacturer_address, manufacturer_contact))

# Generate data for DeviceStaticInfo table
for _ in range(5):  # Assuming you want to generate data for 5 devices
    manufacturer_id = fake.random_int(min=1, max=3)
    model = fake.lexify(text='Model ???')
    purchase_date = fake.date_between(start_date='-3y', end_date='today').strftime('%Y-%m-%d')
    warranty_expiry = fake.date_between(start_date='today', end_date='+3y').strftime('%Y-%m-%d')
    cursor.execute("INSERT INTO DeviceStaticInfo (manufacturer_id, model, purchase_date, warranty_expiry) VALUES (?, ?, ?, ?)",
                   (manufacturer_id, model, purchase_date, warranty_expiry))


# Commit the changes and close the connection
conn.commit()
conn.close()

## Check data

In [ ]:
import mysql.connector

# Establish a connection to the MySQL database
cnx = mysql.connector.connect(user=username, password=password,
                              host=host, database=database)

cursor = cnx.cursor()
sql_str='SELECT * FROM DeviceStaticInfo;'
rs=cursor.execute(sql_str)
rs=cursor.fetchall()
print(rs)

[(1, 1, 'Model bRm', datetime.date(2023, 5, 15), datetime.date(2026, 10, 27)), (2, 1, 'Model lOq', datetime.date(2023, 5, 10), datetime.date(2026, 9, 3)), (3, 3, 'Model idC', datetime.date(2021, 7, 11), datetime.date(2024, 9, 1)), (4, 3, 'Model XAy', datetime.date(2023, 11, 4), datetime.date(2025, 5, 1)), (5, 2, 'Model jAJ', datetime.date(2022, 1, 2), datetime.date(2024, 12, 15))]


In [ ]:
import mysql.connector

# Establish a connection to the MySQL database
cnx = mysql.connector.connect(user=username, password=password,
                              host=host, database=database)

cursor = cnx.cursor()
cursor.execute("SHOW TABLES")

tables = [table[0] for table in cursor.fetchall()]


for table in tables:
    try:
        print(f"Reading data from table: {table}")
        cursor.execute(f"SELECT * FROM {table}")
        rows = cursor.fetchall()

        # Print rows if there are any, else indicate the table is empty
        if rows:
            print(f"Data from {table}:")
            for row in rows:
                print(row)
        else:
            print(f"No data found in {table}.")

    except pymysql.MySQLError as e:
        print(f"Error reading from {table}: {e}")

# Close the connection
cursor.close()
connection.close()

Reading data from table: AlertLifecycle
No data found in AlertLifecycle.
Reading data from table: Alerts
No data found in Alerts.
Reading data from table: DeviceOperationalData
No data found in DeviceOperationalData.
Reading data from table: DeviceStaticInfo
Data from DeviceStaticInfo:
(1, 1, 'Model bRm', datetime.date(2023, 5, 15), datetime.date(2026, 10, 27))
(2, 1, 'Model lOq', datetime.date(2023, 5, 10), datetime.date(2026, 9, 3))
(3, 3, 'Model idC', datetime.date(2021, 7, 11), datetime.date(2024, 9, 1))
(4, 3, 'Model XAy', datetime.date(2023, 11, 4), datetime.date(2025, 5, 1))
(5, 2, 'Model jAJ', datetime.date(2022, 1, 2), datetime.date(2024, 12, 15))
Reading data from table: GlucoseReadings
Data from GlucoseReadings:
(1226, 1, 5, 164.0, datetime.datetime(2023, 11, 3, 8, 17, 18), 'Morganborough')
(1227, 1, 4, 158.31, datetime.datetime(2023, 11, 21, 2, 25, 31), 'East Stacy')
(1228, 1, 3, 127.9, datetime.datetime(2023, 11, 4, 22, 42, 23), 'North George')
(1229, 1, 5, 155.32, datetim

NameError: ignored